In [ ]:
import pandas as pd
#df = pd.read_parquet('sources/metadata.parquet', engine='pyarrow')
#df_large = pd.read_parquet('sources/metadata-large.parquet', engine='pyarrow')

exact_matches = pd.read_parquet("../results/artists_exact_match_large.parquet", engine='pyarrow')
assert "artists" in exact_matches.columns, f'artists is not in {exact_matches.columns}'
assert "num_artists" in exact_matches.columns, f'num_artists is not in {exact_matches.columns}'


# read excel_artist_names
import my_utils

excel_artist_names = my_utils.read_lines_as_list("../sources/excel_artists_copy_paste_name.txt")
hundred_artist_names = excel_artist_names[0:100]
ten_artist_names = excel_artist_names[0:10]


artist_mentions = pd.read_parquet('../results/artist_mentions.parquet', engine='pyarrow')

# Co-occurrences

## For prompts with 2 or more mentioned painters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

artist_mentions.sort_values("mentions", axis=0, ascending=False, inplace=True)
painter_names = artist_mentions.loc[artist_mentions["mentions"] > 0 ]

painter_names["artist"] = painter_names["artist"].apply(lambda x: x.lower()) 
print(f'{len(painter_names)} artists with more than 0 mentions: {painter_names}')
painter_names = painter_names["artist"].to_list()
# painters with more than one mention

# remove this restriction later (or do the analysis only for the top x artists)
painter_names = painter_names[0:10]


cols = ["prompt", "image_name"]

occurences =  pd.DataFrame(columns=cols)
for name in painter_names:
    # create new column for each painter with default value 0
    #occurences[painter_names] = 0
    occurences[name] = pd.Series(dtype=np.float64)
    occurences = occurences.copy()

print("occurences")
print(occurences.columns)

print(f'exact_matches columns {exact_matches.columns}')


cp = exact_matches.copy()

t = cp["artists"].apply(lambda x: pd.Series([1 for _ in range(len(set(painter_names).intersection(x)))], index=set(painter_names).intersection(x), dtype=np.float64))#.copy()#, axis=1)
t = t.fillna(0)


print(f'T shape {t.shape}')
print(t.head())

print("correlation:")
# https://www.w3resource.com/pandas/dataframe/dataframe-cov.php
corr = t.corr()
print(corr)

import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True)
plt.show()

plt.matshow(corr)
plt.show()

print("covariance:")
# https://www.w3resource.com/pandas/dataframe/dataframe-cov.php
cov = t.cov()
print(cov)

plt.matshow(cov)
plt.show()

plt.figure(figsize=(8, 6))
sns.heatmap(cov, annot=True)
plt.show()

print(("extracting finished"))

# Visualising corr and cov matrix
# https://www.delftstack.com/howto/python-pandas/pandas-correlation-matrix/?utm_content=cmp-true


In [ ]:
# Visualize as a network
# https://towardsdatascience.com/tutorial-network-visualization-basics-with-networkx-and-plotly-and-a-little-nlp-57c9bbb55bb9
# take the higher correlations as to be edges in an adjacency matrix

import plotly.offline as py
import plotly.graph_objects as go
import networkx as nx


am_cp = artist_mentions.copy()
am_cp["artist"] = am_cp["artist"].str.lower()
am_cp = am_cp.set_index("artist")

print(am_cp.head(len(painter_names)))

network_positive = nx.Graph()
for pn in painter_names:
    #print(pn)
    #print(am_cp.loc[pn].mentions)
    network_positive.add_node(pn, size = am_cp.loc[pn].mentions)

for pn1 in painter_names:
    for pn2 in painter_names:
        if pn1 != pn2:
            #print(f'corr between {pn1} and {pn2} is {corr[pn1][pn2]}')
            network_positive.add_edge(pn1, pn2, weight = corr[pn1][pn2])

pos_ = nx.spring_layout(network_positive)

# Custom function to create an edge between node x and node y, with a given text and width
def make_edge(x, y, text, width, positive):
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = 'cornflowerblue' if positive else 'red'),
                       hoverinfo = 'text',
                       text      = ([text]),
                       mode      = 'lines')

# For each edge, make an edge_trace, append to list
edge_trace = []
for edge in network_positive.edges():
    #print(f'Edge {edge} with weight {network_positive.edges()[edge]["weight"]}')
    if network_positive.edges()[edge]['weight'] > 0:
        char_1 = edge[0]
        char_2 = edge[1]
        x0, y0 = pos_[char_1]
        x1, y1 = pos_[char_2]
        text   = char_1 + '--' + char_2 + ': ' + str(network_positive.edges()[edge]['weight'])
        #print(f'weight {network_positive.edges()[edge]["weight"]}')
        trace  = make_edge([x0, x1, None], [y0, y1, None], text, 
                           width = network_positive.edges()[edge]['weight'] * 10, positive=True) #**1.75)
        edge_trace.append(trace)
    if network_positive.edges()[edge]['weight'] < 0:
        char_1 = edge[0]
        char_2 = edge[1]
        x0, y0 = pos_[char_1]
        x1, y1 = pos_[char_2]
        text   = char_1 + '--' + char_2 + ': ' + str(- network_positive.edges()[edge]['weight'])
        #print(f'weight {network_positive.edges()[edge]["weight"]}')
        trace  = make_edge([x0, x1, None], [y0, y1, None], text, 
                           width = - network_positive.edges()[edge]['weight'] * 10, positive=False) #**1.75)
        edge_trace.append(trace)


# Make a node trace
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 10,
                        mode      = 'markers+text',
                        hoverinfo = 'none',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))
print("nodes")
print(network_positive.nodes())

max_size = 0
for node in network_positive.nodes():
    if network_positive.nodes()[node]['size'] > max_size:
        max_size = network_positive.nodes()[node]['size']

# For each node in network_positive, get the position and size and add to the node_trace
for node in network_positive.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    #print(f'node {node}')
    #print(network_positive.nodes()[node])
    #print(network_positive.nodes()[node]['size'])
    node_trace['marker']['size'] += tuple([network_positive.nodes()[node]['size']/max_size * 100])
    node_trace['text'] += tuple(['<b>' + node + '</b>'])

    
    # Customize layout
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)', # transparent background
    plot_bgcolor='rgba(0,0,0,0)', # transparent 2nd background
    xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
    yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
)
# Create figure
fig = go.Figure(layout = layout)
# Add all edge traces
for trace in edge_trace:
    fig.add_trace(trace)
#print("node trace")
#print(node_trace)    
# Add node trace
fig.add_trace(node_trace)
# Remove legend
fig.update_layout(showlegend = False)
# Remove tick labels
fig.update_xaxes(showticklabels = False)
fig.update_yaxes(showticklabels = False)

fig.write_html("artist_correlation_network.html")

print("network saved to file")

In [ ]:
import time

c = 0
starttime = time.time()
for _, entry in cp.iterrows():
    #print(entry)
    new_row = dict()
    new_row['image_name'] = entry['image_name']
    new_row['prompt'] = entry['prompt']
    #for name in painter_names:
    #    new_row[name] = 0
    
    # set to one for each painter that really occured
    for name in entry['artists']:
        if name in painter_names:
            new_row[name] = 1
    
    new_row = pd.Series(new_row)
    #print(new_row)
    
    occurences = pd.concat([occurences, new_row.to_frame().T], ignore_index=True)#.copy()
    # copy to defragment the dataframe
    
    c+=1
    if c% 10000 == 0:
        duration = time.time() - starttime
        print(f'{c} {duration / 60} minutes passed, {duration/c} seconds per entry, approximately {duration/c * (cp.shape[0] - c) / 60 / 60} hours remaining')
    
occurences.fillna(0)